# Tensorflow Extended (TFX)
- In this kernel we try to use Tensorflow data validation library to easily visualize the statistics and given schema,the other main purpose of this tool is to compare train and test statistics (Train -Test Skew)
- This tensorflow data validation packages uses Facets internally to render interactive visualization
- This packages was used in production system to validate the data for anomalies before passing into 
    our model (ML/DL model -since model expects data in particular schema)
- For further information kindly go through the **TFX documentation** 

[TFX](https://www.tensorflow.org/tfx)

[Tensorflow Data validation](https://www.tensorflow.org/tfx/data_validation/get_started)


*
Disclaimer: we can analyse the Descriptive nature of the data by rich set of tools already available in the python 
ecosystem, this kernel purpose is to introduce the new tool (from Tensorflow ecosytem) which can be useful in analysing descriptive nature and production system also .*

In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['dipole_moments.csv', 'potential_energy.csv', 'magnetic_shielding_tensors.csv', 'structures.csv', 'scalar_coupling_contributions.csv', 'structures', 'sample_submission.csv', 'mulliken_charges.csv', 'test.csv', 'train.csv']


# Installing Tensorflow data validation
- Tfdv uses Apache Beam to full pass the dataset (Big datasets) and for aggregations 
- Apache Beam has some nice characteristics such as 
    - **unfied programing model **for Batch and stream processing
    - **portable** Execute pipelines on multiple execution environments.
    - Apache beam supports different language SDKs
    - Apache beam runs with different backend runners such as 
        - Apache Flink
        - Apache Spark
        - Apache Gearpump
        - Apache Nemo
        - Apache Samza
        - Google cloud dataflow
        - Direct runners (without parallelism in local machines)
        
*Note this kernel uses Direct runners this may be slow while getting statistics from the dataset.***

In [3]:
!pip install --user tensorflow_data_validation

     |████████████████████████████████| 2.7MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 174kB 40.3MB/s eta 0:00:01
     |████████████████████████████████| 2.8MB 36.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 47.1MB/s eta 0:00:01
     |████████████████████████████████| 225kB 38.4MB/s eta 0:00:01
     |████████████████████████████████| 48.5MB 75.0MB/s eta 0:00:01    |█████▌                          | 8.3MB 10.2MB/s eta 0:00:04
     |████████████████████████████████| 92kB 24.0MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 38.2MB/s eta 0:00:01
     |████████████████████████████████| 61kB 17.8MB/s eta 0:00:01
     |████████████████████████████████| 51kB 21.7MB/s eta 0:00:01
     |████████████████████████████████| 81kB 25.7MB/s eta 0:00:01
     |████████████████████████████████| 276kB 40.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 28.7MB/s ta 0:00:01
     |████████████████████████████████| 92kB 32.4MB/s eta 0:00:01
   

     |████████████████████████████████| 92kB 24.7MB/s eta 0:00:01
  Stored in directory: /tmp/.cache/pip/wheels/fa/8e/22/bf6c72aa7a68be71b39e7a2b7b08dfcde282dc58fa2481047f
  Stored in directory: /tmp/.cache/pip/wheels/6a/a5/14/25541ebcdeaf97a37b6d05c7ff15f5bd20f5e91b99d313e5b4
  Stored in directory: /tmp/.cache/pip/wheels/6d/41/4b/2b369d6e2b7eaebcdd423516d3fb659c7658c16a2be8fd04ec
  Stored in directory: /tmp/.cache/pip/wheels/b1/3c/45/8a3fff4129d91ff533c3e0db9cfbbb70dcb353584e22989574
  Stored in directory: /tmp/.cache/pip/wheels/50/24/4d/4580ca4a299f1ad6fd63443e6e584cb21e9a07988e4aa8daac
  Stored in directory: /tmp/.cache/pip/wheels/fe/a7/05/23e3699975fc20f8a30e00ac1e515ab8c61168e982abe4ce70
  Stored in directory: /tmp/.cache/pip/wheels/48/f7/87/b932f09c6335dbcf45d916937105a372ab14f353a9ca431d7d
  Stored in directory: /tmp/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
  Stored in directory: /tmp/.cache/pip/wheels/d6/c2/92/837e8a4d649a209dff85b38d7fbb576

ERROR: google-cloud-storage 1.16.1 has requirement google-cloud-core<2.0dev,>=1.0.0, but you'll have google-cloud-core 0.29.1 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import tensorflow_data_validation as tfdv

/root/.local/lib/python3.6/site-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


# Generating Statistics
Satistics is nothing but tfdv full passes the dataset and collects the characteristics (aggregation metrics) of 
the dataset and use this statistics (Train statistics ) to validate the statistics of the test data in prediction or serving time.
- Currently, We can prepare Satistics of the dataset in three ways :
    - Directly from the CSVs
    - From pandas Dataframe
    - From Tensorflow specific- TFRecord

In [5]:
dipole_moments=pd.read_csv("../input/dipole_moments.csv")
magnetic_shielding_tensors=pd.read_csv("../input/magnetic_shielding_tensors.csv")
mulliken_charges=pd.read_csv("../input/mulliken_charges.csv")
potential_energy=pd.read_csv("../input/potential_energy.csv")
structures=pd.read_csv("../input/structures.csv")
scalar_coupling_contributions=pd.read_csv("../input/scalar_coupling_contributions.csv")
#dipole_moments=pd.read_csv("../input/dipole_moments.csv")


In [ ]:
#dipole_moments=tfdv.generate_statistics_from_csv('../input/dipole_moments.csv')
#magnetic_shielding_tensors=tfdv.generate_statistics_from_csv('../input/magnetic_shielding_tensors.csv')
#mulliken_charges=tfdv.generate_statistics_from_csv('../input/mulliken_charges.csv')
#potential_energy=tfdv.generate_statistics_from_csv('../input/potential_energy.csv')
#scalar_coupling_contributions=tfdv.generate_statistics_from_csv('../input/scalar_coupling_contributions.csv')

Run any of the above cells to generate statistics, i observed generating statistics from the pandas dataframe was 
bit faster than generating from the CSVs.

# Visualizing the Generated Statistics in Facets 
- This is as simple as calling **tfdv.visualize_statistics** api

In [6]:
dipole_moments_stats=tfdv.generate_statistics_from_dataframe(dipole_moments)
tfdv.visualize_statistics(dipole_moments_stats)

In [7]:
magnetic_shielding_tensors_stats=tfdv.generate_statistics_from_dataframe(magnetic_shielding_tensors)
tfdv.visualize_statistics(magnetic_shielding_tensors_stats)

In [8]:
mulliken_charges_stats=tfdv.generate_statistics_from_dataframe(mulliken_charges)
tfdv.visualize_statistics(mulliken_charges_stats)

In [9]:
potential_energy_stats=tfdv.generate_statistics_from_dataframe(potential_energy)
tfdv.visualize_statistics(potential_energy_stats)

In [10]:
structures_stats=tfdv.generate_statistics_from_dataframe(structures)
tfdv.visualize_statistics(structures_stats)

In [11]:
scalar_coupling_contributions_stats=tfdv.generate_statistics_from_dataframe(scalar_coupling_contributions)
tfdv.visualize_statistics(scalar_coupling_contributions_stats)

# Saving schema for training statistics

- Schema is nothing but the expected features /schema of the given dataset
    - Schema can be prepared in two ways:
        - Manually write the schema
        - USe the generated Satistics to infer schema.
- You can save/serialize the schema of your training dataset in to the disk and later you load and validate 
    the statistis of the test data

In [20]:
dipole_moments_schema=tfdv.infer_schema(dipole_moments_stats)
tfdv.write_schema_text(dipole_moments_schema,"dipole_moments_schema")

magnetic_shielding_tensors_schema=tfdv.infer_schema(magnetic_shielding_tensors_stats)
tfdv.write_schema_text(magnetic_shielding_tensors_schema,"magnetic_shielding_tensors_schema")

mulliken_charges_schema=tfdv.infer_schema(mulliken_charges_stats)
tfdv.write_schema_text(mulliken_charges_schema,"mulliken_charges_schema")

potential_energy_schema=tfdv.infer_schema(potential_energy_stats)
tfdv.write_schema_text(potential_energy_schema,"potential_energy_schema")

structures_schema=tfdv.infer_schema(structures_stats)
tfdv.write_schema_text(structures_schema,"structures_schema")

scalar_coupling_contributions_schema=tfdv.infer_schema(scalar_coupling_contributions_stats)
tfdv.write_schema_text(scalar_coupling_contributions_schema,"scalar_coupling_contributions_schema")


Schemas are stored in Output DIR = '.'

In [27]:
print(os.listdir(".")),tfdv.load_schema_text('magnetic_shielding_tensors_schema')

['dipole_moments_schema', 'mulliken_charges_schema', 'magnetic_shielding_tensors_schema', 'structures_schema', '__notebook_source__.ipynb', '.ipynb_checkpoints', 'potential_energy_schema', 'scalar_coupling_contributions_schema']


(None, feature {
   name: "molecule_name"
   type: BYTES
   presence {
     min_fraction: 1.0
     min_count: 1
   }
   shape {
     dim {
       size: 1
     }
   }
 }
 feature {
   name: "atom_index"
   type: INT
   presence {
     min_fraction: 1.0
     min_count: 1
   }
   shape {
     dim {
       size: 1
     }
   }
 }
 feature {
   name: "XX"
   type: FLOAT
   presence {
     min_fraction: 1.0
     min_count: 1
   }
   shape {
     dim {
       size: 1
     }
   }
 }
 feature {
   name: "YX"
   type: FLOAT
   presence {
     min_fraction: 1.0
     min_count: 1
   }
   shape {
     dim {
       size: 1
     }
   }
 }
 feature {
   name: "ZX"
   type: FLOAT
   presence {
     min_fraction: 1.0
     min_count: 1
   }
   shape {
     dim {
       size: 1
     }
   }
 }
 feature {
   name: "XY"
   type: FLOAT
   presence {
     min_fraction: 1.0
     min_count: 1
   }
   shape {
     dim {
       size: 1
     }
   }
 }
 feature {
   name: "YY"
   type: FLOAT
   presence {
     min_fr

# Validate Generated Schema
- Look at the generated schema, and validate the data type of the each feature, if everything is allright you can
save the schema and use this schema to validate the test data

In [29]:
tfdv.display_schema(dipole_moments_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'molecule_name',BYTES,required,,-
'X',FLOAT,required,,-
'Y',FLOAT,required,,-
'Z',FLOAT,required,,-


In [30]:
tfdv.display_schema(magnetic_shielding_tensors_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'molecule_name',BYTES,required,,-
'atom_index',INT,required,,-
'XX',FLOAT,required,,-
'YX',FLOAT,required,,-
'ZX',FLOAT,required,,-
'XY',FLOAT,required,,-
'YY',FLOAT,required,,-
'ZY',FLOAT,required,,-
'XZ',FLOAT,required,,-


In [31]:
tfdv.display_schema(mulliken_charges_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'molecule_name',BYTES,required,,-
'atom_index',INT,required,,-
'mulliken_charge',FLOAT,required,,-


In [32]:
tfdv.display_schema(potential_energy_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'molecule_name',BYTES,required,,-
'potential_energy',FLOAT,required,,-


In [33]:
tfdv.display_schema(scalar_coupling_contributions_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'molecule_name',BYTES,required,,-
'atom_index_0',INT,required,,-
'atom_index_1',INT,required,,-
'type',STRING,required,,'type'
'fc',FLOAT,required,,-
'sd',FLOAT,required,,-
'pso',FLOAT,required,,-
'dso',FLOAT,required,,-


,Values
Domain,
'type',"'1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN'"


In [34]:
tfdv.display_schema(structures_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'molecule_name',BYTES,required,,-
'atom_index',INT,required,,-
'atom',STRING,required,,'atom'
'x',FLOAT,required,,-
'y',FLOAT,required,,-
'z',FLOAT,required,,-


,Values
Domain,
'atom',"'C', 'F', 'H', 'N', 'O'"


In [35]:
train=pd.read_csv('../input/train.csv')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [36]:
test=pd.read_csv('../input/test.csv')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [37]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

test = map_atom_info(test, 0)
test = map_atom_info(test, 1)


In [38]:
train.shape,test.shape

((4658147, 14), (2505542, 13))

In [40]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [41]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0


# Lets Generate Statistics for Train and Test datasets 
- This time use all the train features in single dataframe and generate statistics 
- Use the above generated Train statistics to the test datasets to identify anomalies or data deviations

In [ ]:
train_stats=tfdv.generate_statistics_from_dataframe(train)

In [ ]:
test_stats=tfdv.generate_statistics_from_dataframe(test)

In [ ]:
train_schema=tfdv.infer_schema(train_stats)
tfdv.write_schema_text(train_schema)

In [ ]:
test_schema=tfdv.infer_schema(test_stats)
tfdv.write_schema_text(test_schema)

# Validation

In [ ]:
tfdv.validate_statistics()